In [74]:
import numpy as np
import pandas as pd
from datetime import timedelta
import Levenshtein
pd.options.mode.chained_assignment = None

## Problem statement
For each state, district and overall, find the following ratios: total number of females vaccinated (either 1 or 2 doses) to total number of males vaccinated (same). For that district/state/country, find the ratio of population of females to males. (If a district is absent in 2011 census, drop it from analysis.) Now find the ratio of the two ratios, i.e., vaccination ratio to population ratio.
Output them in the following manner: districtid, vaccinationratio, populationratio, ratioofratios.

In [75]:
vacc_data1 = pd.read_csv('../data/cowin_vaccine_data_districtwise.csv', low_memory=False)
vacc_data1

,S No,State_Code,State,District_Key,Cowin Key,District,16/01/2021,16/01/2021.1,16/01/2021.2,16/01/2021.3,...,31/10/2021,31/10/2021.1,31/10/2021.2,31/10/2021.3,31/10/2021.4,31/10/2021.5,31/10/2021.6,31/10/2021.7,31/10/2021.8,31/10/2021.9
0,NaN,NaN,NaN,NaN,NaN,NaN,Total Individuals Registered,Sessions,Sites,First Dose Administered,...,Total Doses Administered,Sessions,Sites,First Dose Administered,Second Dose Administered,Male(Individuals Vaccinated),Female(Individuals Vaccinated),Transgender(Individuals Vaccinated),Covaxin (Doses Administered),CoviShield (Doses Administered)
1,1.0,AN,Andaman and Nicobar Islands,AN_Nicobars,Nicobar,Nicobars,745,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,AN,Andaman and Nicobar Islands,AN_North and Middle Andaman,North and Middle Andaman,North and Middle Andaman,1496,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,AN,Andaman and Nicobar Islands,AN_South Andaman,South Andaman,South Andaman,6028,2,2,23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,AP,Andhra Pradesh,AP_Anantapur,Anantapur,Anantapur,20781,28,26,287,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5.0,AP,Andhra Pradesh,AP_Chittoor,Chittoor,Chittoor,26285,63,31,424,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6.0,AP,Andhra Pradesh,AP_East Godavari,East Godavari,East Godavari,23819,39,37,1012,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7.0,AP,Andhra Pradesh,AP_Guntur,Guntur,Guntur,25859,31,31,303,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8.0,AP,Andhra Pradesh,AP_Krishna,Krishna,Krishna,31419,42,31,144,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9.0,AP,Andhra Pradesh,AP_Kurnool,Kurnool,Kurnool,18219,54,29,214,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
vacc_data1.iloc[0]

S No                                            NaN
State_Code                                      NaN
State                                           NaN
District_Key                                    NaN
Cowin Key                                       NaN
District                                        NaN
16/01/2021             Total Individuals Registered
16/01/2021.1                               Sessions
16/01/2021.2                                 Sites 
16/01/2021.3                First Dose Administered
16/01/2021.4               Second Dose Administered
16/01/2021.5           Male(Individuals Vaccinated)
16/01/2021.6         Female(Individuals Vaccinated)
16/01/2021.7    Transgender(Individuals Vaccinated)
16/01/2021.8           Covaxin (Doses Administered)
16/01/2021.9        CoviShield (Doses Administered)
17/01/2021             Total Individuals Registered
17/01/2021.1                               Sessions
17/01/2021.2                                 Sites 
17/01/2021.3

Few Districts in CoWin data has been repeated, merging them into one. For example, for Ahmedabad and Ahmedabad corporation, merged into Ahmedabad.

In [77]:
vacc_data = vacc_data1.iloc[1:]
vacc_data.fillna(0, inplace=True)
col = list(vacc_data.columns[6:])
vacc_data[col] = vacc_data[col].astype(int)
vacc_data

,S No,State_Code,State,District_Key,Cowin Key,District,16/01/2021,16/01/2021.1,16/01/2021.2,16/01/2021.3,...,31/10/2021,31/10/2021.1,31/10/2021.2,31/10/2021.3,31/10/2021.4,31/10/2021.5,31/10/2021.6,31/10/2021.7,31/10/2021.8,31/10/2021.9
1,1.0,AN,Andaman and Nicobar Islands,AN_Nicobars,Nicobar,Nicobars,745,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2.0,AN,Andaman and Nicobar Islands,AN_North and Middle Andaman,North and Middle Andaman,North and Middle Andaman,1496,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3.0,AN,Andaman and Nicobar Islands,AN_South Andaman,South Andaman,South Andaman,6028,2,2,23,...,0,0,0,0,0,0,0,0,0,0
4,4.0,AP,Andhra Pradesh,AP_Anantapur,Anantapur,Anantapur,20781,28,26,287,...,0,0,0,0,0,0,0,0,0,0
5,5.0,AP,Andhra Pradesh,AP_Chittoor,Chittoor,Chittoor,26285,63,31,424,...,0,0,0,0,0,0,0,0,0,0
6,6.0,AP,Andhra Pradesh,AP_East Godavari,East Godavari,East Godavari,23819,39,37,1012,...,0,0,0,0,0,0,0,0,0,0
7,7.0,AP,Andhra Pradesh,AP_Guntur,Guntur,Guntur,25859,31,31,303,...,0,0,0,0,0,0,0,0,0,0
8,8.0,AP,Andhra Pradesh,AP_Krishna,Krishna,Krishna,31419,42,31,144,...,0,0,0,0,0,0,0,0,0,0
9,9.0,AP,Andhra Pradesh,AP_Kurnool,Kurnool,Kurnool,18219,54,29,214,...,0,0,0,0,0,0,0,0,0,0
10,10.0,AP,Andhra Pradesh,AP_Prakasam,Prakasam,Prakasam,13015,17,16,202,...,0,0,0,0,0,0,0,0,0,0


In [78]:
distr_data = pd.read_csv('../output/edge-graph.csv', index_col='District_Key')
distr_data.drop(columns=['Unnamed: 0'], inplace=True)
districts_key = list(distr_data.index)
districts_key

['AP_Anantapur',
 'AP_Chittoor',
 'AP_East Godavari',
 'AP_Guntur',
 'AP_Krishna',
 'AP_Kurnool',
 'AP_Prakasam',
 'AP_S.P.S. Nellore',
 'AP_Srikakulam',
 'AP_Visakhapatnam',
 'AP_Vizianagaram',
 'AP_West Godavari',
 'AP_Y.S.R. Kadapa',
 'AR_Anjaw',
 'AR_Changlang',
 'AR_East Kameng',
 'AR_East Siang',
 'AR_Kamle',
 'AR_Kra Daadi',
 'AR_Kurung Kumey',
 'AR_Lepa Rada',
 'AR_Lohit',
 'AR_Longding',
 'AR_Lower Dibang Valley',
 'AR_Lower Siang',
 'AR_Lower Subansiri',
 'AR_Namsai',
 'AR_Pakke Kessang',
 'AR_Papum Pare',
 'AR_Shi Yomi',
 'AR_Siang',
 'AR_Tawang',
 'AR_Tirap',
 'AR_Upper Dibang Valley',
 'AR_Upper Siang',
 'AR_Upper Subansiri',
 'AR_West Kameng',
 'AR_West Siang',
 'AS_Baksa',
 'AS_Barpeta',
 'AS_Biswanath',
 'AS_Bongaigaon',
 'AS_Cachar',
 'AS_Charaideo',
 'AS_Chirang',
 'AS_Darrang',
 'AS_Dhemaji',
 'AS_Dhubri',
 'AS_Dibrugarh',
 'AS_Dima Hasao',
 'AS_Goalpara',
 'AS_Golaghat',
 'AS_Hailakandi',
 'AS_Hojai',
 'AS_Jorhat',
 'AS_Kamrup',
 'AS_Kamrup Metropolitan',
 'AS_Karbi

In [79]:
map_dkey_sname, map_dkey_dname = {}, {}
for dt in districts_key:
    map_dkey_sname[dt] = vacc_data[vacc_data['District_Key'] == dt]['State'].iloc[0]
    map_dkey_dname[dt] = vacc_data[vacc_data['District_Key'] == dt]['District'].iloc[0]
map_dkey_dname

{'AP_Anantapur': 'Anantapur',
 'AP_Chittoor': 'Chittoor',
 'AP_East Godavari': 'East Godavari',
 'AP_Guntur': 'Guntur',
 'AP_Krishna': 'Krishna',
 'AP_Kurnool': 'Kurnool',
 'AP_Prakasam': 'Prakasam',
 'AP_S.P.S. Nellore': 'S.P.S. Nellore',
 'AP_Srikakulam': 'Srikakulam',
 'AP_Visakhapatnam': 'Visakhapatnam',
 'AP_Vizianagaram': 'Vizianagaram',
 'AP_West Godavari': 'West Godavari',
 'AP_Y.S.R. Kadapa': 'Y.S.R. Kadapa',
 'AR_Anjaw': 'Anjaw',
 'AR_Changlang': 'Changlang',
 'AR_East Kameng': 'East Kameng',
 'AR_East Siang': 'East Siang',
 'AR_Kamle': 'Kamle',
 'AR_Kra Daadi': 'Kra Daadi',
 'AR_Kurung Kumey': 'Kurung Kumey',
 'AR_Lepa Rada': 'Lepa Rada',
 'AR_Lohit': 'Lohit',
 'AR_Longding': 'Longding',
 'AR_Lower Dibang Valley': 'Lower Dibang Valley',
 'AR_Lower Siang': 'Lower Siang',
 'AR_Lower Subansiri': 'Lower Subansiri',
 'AR_Namsai': 'Namsai',
 'AR_Pakke Kessang': 'Pakke Kessang',
 'AR_Papum Pare': 'Papum Pare',
 'AR_Shi Yomi': 'Shi Yomi',
 'AR_Siang': 'Siang',
 'AR_Tawang': 'Tawang'

In [80]:
pop_data = pd.read_excel('../data/DDW_PCA0000_2011_Indiastatedist.xlsx')
pop_data = pop_data[['Name', 'Level', 'TRU', 'TOT_M', 'TOT_F']]
pop_data = pop_data[pop_data['TRU'] == 'Total']
pop_data.drop(columns=['TRU'], inplace=True)
pop_data

,Name,Level,TOT_M,TOT_F
0,India,India,623270258,587584719
3,JAMMU & KASHMIR,STATE,6640662,5900640
6,Kupwara,DISTRICT,474190,396164
9,Badgam,DISTRICT,398041,355704
12,Leh(Ladakh),DISTRICT,78971,54516
15,Kargil,DISTRICT,77785,63017
18,Punch,DISTRICT,251899,224936
21,Rajouri,DISTRICT,345351,297064
24,Kathua,DISTRICT,326109,290326
27,Baramula,DISTRICT,534733,473306


In [81]:
overall_pop_data = pop_data[pop_data['Level'] == 'India']
overall_pop_data
# vacc_data[vacc_data['State'] == 'Dadra and Nagar Haveli and Daman and Diu']

,Name,Level,TOT_M,TOT_F
0,India,India,623270258,587584719


In [82]:
# This function will find the most similar district word from the given district word in the dataset.
def get_most_similar_word(df, w, name):
    '''
    inputs:
        df : input data containing all the state and districts code i.e DataFrame format
        w : district name
    output:
        output1 : most similar district
        output2 : similarity score between two given word input and most similar word
        
    '''
    temp_df = df.copy()
    temp_df = temp_df[1:]
    temp_df['Levenshtein_dist'] = temp_df[name].apply(lambda x: Levenshtein.distance(w, x))
    temp_df = temp_df.sort_values(by=['Levenshtein_dist'])
#     print(temp_df['District'].iloc[0])
    #print("with value: ", temp_df['Levenshtein_dist'].iloc[0])
    return temp_df[name].iloc[0], temp_df['Levenshtein_dist'].iloc[0]

In [83]:
#Manually created correct match from population datasets.
def find_correct_match(distr):
    '''
    inputs:
        distr : district name for finding correct match if  Levenshtein_distance doesn't work
    output:
        output1 : correct match if available otherwise return empty string
        
    '''
    correct_match = {'Belgaum': 'Belagavi', 'Leh(Ladakh)':'Leh', 'Gurgaon':'Gurugram', 'Baleshwar': 'Balasore', 'Rangareddy': 'Ranga Reddy', 'Hugli':'Hooghly', 'Sahibzada Ajit Singh Nagar': 'S.A.S. Nagar', 
                 'Sant Ravidas Nagar':'Bhadohi','Faizabad':'Ayodhya', 'Muktsar': 'Sri Muktsar Sahib', 'Pashchim Champaran':'West Champaran','Pashchimi Singhbhum': 'West Singhbhum',
                'Bid': 'Beed', 'East Karbi Anglong':'Karbi Anglong', 'The Dangs':'Dang', 'Dantewada':'Dakshin Bastar Dantewada','Purbi Singhbhum':'East Singhbhum', 'Warangal':'Warangal Urban',
                'Jyotiba Phule Nagar': 'Amroha', 'Allahabad ':'Prayagraj', 'Khandwa (East Nimar)':'Khandwa', 'Dohad':'Dahod', 'Kachchh':'Kutch', 'Kheri':'Lakhimpur Kheri', 'Kaimur (Bhabua)':'Kaimur', 
                'Y.S.R.':'Y.S.R. Kadapa', "Sant Ravidas Nagar (Bhadohi)": "Bhadohi", 'Gulbarga':'Kalaburagi', 'Shimoga':'Shivamogga', 'Bangalore':'Bengaluru Urban'}
    if distr in correct_match.keys():
        return correct_match[distr]
    return ''

In [84]:
#Not including this district 
def not_included_district(dt):
    not_included = ['Mewat ', 'North West', 'North', 'North East', 'East', 
                    'Central', 'West', 'South West', 'South',
                    'Kanshiram Nagar', 'North  District','West District', 'Barddhaman ',
                    'South District', 'East District', 'Mumbai Suburban', 'Mahamaya Nagar']
    
    if dt not in not_included: return True
    return False

In [85]:
df = pop_data.copy()
df = df[df['Level'] == 'DISTRICT']
map_dt_cor_dt={}
map_dname_dkey={}
for dt in df['Name'].unique():
    if not not_included_district(dt): 
        print("Not including this district: ", dt)
        continue
    if dt not in map_dkey_dname.values():
        sm_dt, score = get_most_similar_word(vacc_data, dt, 'District')
        cor_match_dt = find_correct_match(dt)
        if len(cor_match_dt)>0:
            sm_dt = cor_match_dt
            map_dt_cor_dt[dt] = sm_dt
            map_dname_dkey[dt] = vacc_data[vacc_data['District'] == sm_dt]['District_Key'].iloc[0]
            print("District available: ", dt)
            print("Correct Matched found with: ", cor_match_dt)
        elif sm_dt in map_dkey_dname.values():
            print("District available: ", dt)
            print("Correct Matched with: ", sm_dt)
            map_dt_cor_dt[dt] = sm_dt
            map_dname_dkey[dt] = vacc_data[vacc_data['District'] == sm_dt]['District_Key'].iloc[0]
    else:
        map_dt_cor_dt[dt] = dt
        map_dname_dkey[dt] = vacc_data[vacc_data['District'] == dt]['District_Key'].iloc[0]

df['District'] = df['Name'].apply(lambda x: map_dt_cor_dt[x] if x in map_dt_cor_dt.keys() else np.nan)
df.dropna(subset=['District'], inplace=True)
df['District_Key'] = df['Name'].apply(lambda x: map_dname_dkey[x])
df.drop(columns=['Name', 'District', 'Level'], inplace=True)
df = df[['District_Key', 'TOT_M', 'TOT_F']]

District available:  Badgam
Correct Matched with:  Budgam
District available:  Leh(Ladakh)
Correct Matched found with:  Leh
District available:  Baramula
Correct Matched with:  Baramulla
District available:  Bandipore
Correct Matched with:  Bandipora
District available:  Shupiyan
Correct Matched with:  Shopiyan
District available:  Lahul & Spiti
Correct Matched with:  Lahaul and Spiti
District available:  Kapurthala 
Correct Matched with:  Kapurthala
District available:  Shahid Bhagat Singh Nagar 
Correct Matched with:  Shahid Bhagat Singh Nagar
District available:  Firozpur
Correct Matched with:  Ferozepur
District available:  Muktsar
Correct Matched found with:  Sri Muktsar Sahib
District available:  Amritsar 
Correct Matched with:  Amritsar
District available:  Sahibzada Ajit Singh Nagar
Correct Matched found with:  S.A.S. Nagar
District available:  Garhwal
Correct Matched with:  Garhwa
District available:  Almora

Correct Matched with:  Almora
District available:  Hardwar
Correct M

District available:  Bangalore Rural
Correct Matched with:  Bengaluru Rural
District available:  Idukki 
Correct Matched with:  Idukki
District available:  Namakkal   
Correct Matched with:  Namakkal
District available:  The Nilgiris
Correct Matched with:  Nilgiris
District available:  Karur 
Correct Matched with:  Karur
District available:  Perambalur  
Correct Matched with:  Perambalur
District available:  Ariyalur  
Correct Matched with:  Ariyalur
District available:  Nagapattinam  
Correct Matched with:  Nagapattinam
District available:  Theni  
Correct Matched with:  Theni
District available:  Tirunelveli 
Correct Matched with:  Tirunelveli
District available:  Kanniyakumari
Correct Matched with:  Kanyakumari
District available:  Nicobars
Correct Matched with:  Nalbari


In [86]:
sdf = pop_data.copy()
sdf = sdf[sdf['Level'] == 'STATE']
is_available = {'DAMAN & DIU':'Dadra and Nagar Haveli and Daman and Diu', 
                'DADRA & NAGAR HAVELI': 'Dadra and Nagar Haveli and Daman and Diu'}
for st in sdf['Name'].unique():
    if st in is_available.keys():
        sdf.loc[sdf['Name'] == st, 'State_Code'] = vacc_data[vacc_data['State'] == is_available[st]]['State_Code'].iloc[0]
    else:
        sm_st, _ = get_most_similar_word(vacc_data, st.title(), 'State')
        sdf.loc[sdf['Name'] == st, 'State_Code'] = vacc_data[vacc_data['State'] == sm_st]['State_Code'].iloc[0]
sdf.drop(columns=['Name', 'Level'], inplace=True)
sdf = sdf.groupby('State_Code').sum()
sdf

,TOT_M,TOT_F
State_Code,,
AN,202871,177710
AP,42442146,42138631
AR,713912,669815
AS,15939443,15266133
BR,54278157,49821295
CH,580663,474787
CT,12832895,12712303
DL,8987326,7800615
DN,344061,242895


In [87]:
distr_wise = df.copy()
distr_wise

,District_Key,TOT_M,TOT_F
6,JK_Kupwara,474190,396164
9,JK_Budgam,398041,355704
12,LA_Leh,78971,54516
15,LA_Kargil,77785,63017
18,JK_Punch,251899,224936
21,JK_Rajouri,345351,297064
24,JK_Kathua,326109,290326
27,JK_Baramulla,534733,473306
30,JK_Bandipora,207680,184552
33,JK_Srinagar,651124,585705


In [88]:
ddf = pd.merge(vacc_data, distr_wise, how ='inner', on =['District_Key'])
ddf.fillna(0, inplace=True)
ddf

,S No,State_Code,State,District_Key,Cowin Key,District,16/01/2021,16/01/2021.1,16/01/2021.2,16/01/2021.3,...,31/10/2021.2,31/10/2021.3,31/10/2021.4,31/10/2021.5,31/10/2021.6,31/10/2021.7,31/10/2021.8,31/10/2021.9,TOT_M,TOT_F
0,4.0,AP,Andhra Pradesh,AP_Anantapur,Anantapur,Anantapur,20781,28,26,287,...,0,0,0,0,0,0,0,0,2064495,2016653
1,5.0,AP,Andhra Pradesh,AP_Chittoor,Chittoor,Chittoor,26285,63,31,424,...,0,0,0,0,0,0,0,0,2090204,2083860
2,6.0,AP,Andhra Pradesh,AP_East Godavari,East Godavari,East Godavari,23819,39,37,1012,...,0,0,0,0,0,0,0,0,2569688,2584608
3,7.0,AP,Andhra Pradesh,AP_Guntur,Guntur,Guntur,25859,31,31,303,...,0,0,0,0,0,0,0,0,2440521,2447292
4,8.0,AP,Andhra Pradesh,AP_Krishna,Krishna,Krishna,31419,42,31,144,...,0,0,0,0,0,0,0,0,2267375,2250023
5,9.0,AP,Andhra Pradesh,AP_Kurnool,Kurnool,Kurnool,18219,54,29,214,...,0,0,0,0,0,0,0,0,2039227,2014236
6,10.0,AP,Andhra Pradesh,AP_Prakasam,Prakasam,Prakasam,13015,17,16,202,...,0,0,0,0,0,0,0,0,1714764,1682684
7,11.0,AP,Andhra Pradesh,AP_S.P.S. Nellore,Sri Potti Sriramulu Nellore,S.P.S. Nellore,16944,64,26,277,...,0,0,0,0,0,0,0,0,1492974,1470583
8,12.0,AP,Andhra Pradesh,AP_Srikakulam,Srikakulam,Srikakulam,16335,35,18,165,...,0,0,0,0,0,0,0,0,1341738,1361376
9,13.0,AP,Andhra Pradesh,AP_Visakhapatnam,Visakhapatnam,Visakhapatnam,41378,64,35,527,...,0,0,0,0,0,0,0,0,2138910,2151679


In [89]:
state_wise_pop_data = sdf.copy()
state_wise_pop_data = state_wise_pop_data.reset_index()
state_wise_pop_data

,State_Code,TOT_M,TOT_F
0,AN,202871,177710
1,AP,42442146,42138631
2,AR,713912,669815
3,AS,15939443,15266133
4,BR,54278157,49821295
5,CH,580663,474787
6,CT,12832895,12712303
7,DL,8987326,7800615
8,DN,344061,242895
9,GA,739140,719405


In [90]:
start_date = pd.to_datetime('16/01/2021', format='%d/%m/%Y').date()
end_date = pd.to_datetime('14/08/2021', format='%d/%m/%Y').date()
vacc_col = []
get_original_date=[]
while start_date<=end_date:
    start_date = start_date.strftime('%d/%m/%Y')
    male_vacc_dt = str(start_date)+'.5'
    female_vacc_dt = str(start_date)+'.6'
    
    #Convert back to datetime format
    start_date = start_date.split('.')[0]
    start_date = pd.to_datetime(start_date, format='%d/%m/%Y').date()
    
    if male_vacc_dt not in ddf.columns or female_vacc_dt not in ddf.columns:
        print("Date not available: ", start_date)
        continue
    ddf[male_vacc_dt] = ddf[male_vacc_dt].astype(int)
    ddf[female_vacc_dt] = ddf[female_vacc_dt].astype(int)
    get_original_date.append(start_date.strftime('%d/%m/%Y'))
    vacc_col.append(start_date.strftime('%d/%m/%Y')+'_m')
    vacc_col.append(start_date.strftime('%d/%m/%Y')+'_f')
    ddf[start_date.strftime('%d/%m/%Y')+'_m'] = ddf[male_vacc_dt] 
    ddf[start_date.strftime('%d/%m/%Y')+'_f'] = ddf[female_vacc_dt]
    start_date += timedelta(days=1)

In [91]:
ddf

,S No,State_Code,State,District_Key,Cowin Key,District,16/01/2021,16/01/2021.1,16/01/2021.2,16/01/2021.3,...,10/08/2021_m,10/08/2021_f,11/08/2021_m,11/08/2021_f,12/08/2021_m,12/08/2021_f,13/08/2021_m,13/08/2021_f,14/08/2021_m,14/08/2021_f
0,4.0,AP,Andhra Pradesh,AP_Anantapur,Anantapur,Anantapur,20781,28,26,287,...,842803,1012332,848328,1018210,849753,1019792,888085,1062646,893118,1067715
1,5.0,AP,Andhra Pradesh,AP_Chittoor,Chittoor,Chittoor,26285,63,31,424,...,959035,1144796,959562,1145279,959674,1145347,995271,1189398,1004937,1201195
2,6.0,AP,Andhra Pradesh,AP_East Godavari,East Godavari,East Godavari,23819,39,37,1012,...,1096151,1327508,1097950,1329084,1098676,1329613,1123428,1351663,1125958,1353691
3,7.0,AP,Andhra Pradesh,AP_Guntur,Guntur,Guntur,25859,31,31,303,...,1046575,1209761,1054043,1219950,1057168,1223693,1080897,1253897,1087696,1262966
4,8.0,AP,Andhra Pradesh,AP_Krishna,Krishna,Krishna,31419,42,31,144,...,1058828,1191399,1062976,1195887,1067677,1200824,1078244,1211420,1090145,1223327
5,9.0,AP,Andhra Pradesh,AP_Kurnool,Kurnool,Kurnool,18219,54,29,214,...,710356,948005,713858,952733,715195,954348,731883,975864,744366,990736
6,10.0,AP,Andhra Pradesh,AP_Prakasam,Prakasam,Prakasam,13015,17,16,202,...,774141,918011,783987,928318,786827,931288,793011,938369,800299,947172
7,11.0,AP,Andhra Pradesh,AP_S.P.S. Nellore,Sri Potti Sriramulu Nellore,S.P.S. Nellore,16944,64,26,277,...,761794,878515,763208,880405,773264,892302,776315,895876,783012,903421
8,12.0,AP,Andhra Pradesh,AP_Srikakulam,Srikakulam,Srikakulam,16335,35,18,165,...,581805,767570,584893,771599,586029,772878,587487,774910,588757,776653
9,13.0,AP,Andhra Pradesh,AP_Visakhapatnam,Visakhapatnam,Visakhapatnam,41378,64,35,527,...,1141009,1207486,1159783,1226517,1162259,1228427,1198510,1269414,1213497,1284911


In [92]:
dkey_skey_col = ['State_Code', 'District_Key', 'TOT_M', 'TOT_F']
final_col = dkey_skey_col + vacc_col

final_df = ddf[final_col]
final_df.sort_values('District_Key', inplace=True)
final_df

,State_Code,District_Key,TOT_M,TOT_F,16/01/2021_m,16/01/2021_f,17/01/2021_m,17/01/2021_f,18/01/2021_m,18/01/2021_f,...,10/08/2021_m,10/08/2021_f,11/08/2021_m,11/08/2021_f,12/08/2021_m,12/08/2021_f,13/08/2021_m,13/08/2021_f,14/08/2021_m,14/08/2021_f
0,AP,AP_Anantapur,2064495,2016653,28,259,48,456,69,709,...,842803,1012332,848328,1018210,849753,1019792,888085,1062646,893118,1067715
1,AP,AP_Chittoor,2090204,2083860,93,331,208,804,183,742,...,959035,1144796,959562,1145279,959674,1145347,995271,1189398,1004937,1201195
2,AP,AP_East Godavari,2569688,2584608,283,729,826,1233,1716,1410,...,1096151,1327508,1097950,1329084,1098676,1329613,1123428,1351663,1125958,1353691
3,AP,AP_Guntur,2440521,2447292,39,264,65,407,88,559,...,1046575,1209761,1054043,1219950,1057168,1223693,1080897,1253897,1087696,1262966
4,AP,AP_Krishna,2267375,2250023,69,75,83,146,132,187,...,1058828,1191399,1062976,1195887,1067677,1200824,1078244,1211420,1090145,1223327
5,AP,AP_Kurnool,2039227,2014236,47,167,114,517,104,564,...,710356,948005,713858,952733,715195,954348,731883,975864,744366,990736
6,AP,AP_Prakasam,1714764,1682684,27,175,95,773,153,1142,...,774141,918011,783987,928318,786827,931288,793011,938369,800299,947172
7,AP,AP_S.P.S. Nellore,1492974,1470583,48,229,92,634,100,898,...,761794,878515,763208,880405,773264,892302,776315,895876,783012,903421
8,AP,AP_Srikakulam,1341738,1361376,20,145,33,371,87,442,...,581805,767570,584893,771599,586029,772878,587487,774910,588757,776653
9,AP,AP_Visakhapatnam,2138910,2151679,92,435,123,821,150,1178,...,1141009,1207486,1159783,1226517,1162259,1228427,1198510,1269414,1213497,1284911


In [93]:
def get_districtwise_df(final_df, dt):
    ddf = final_df.copy()
    ddf = ddf[ddf['District_Key'] == dt]

    d1 = [c for c in ddf.columns[4::2]]
    d2 = [c for c in ddf.columns[5::2]]
    male_df = ddf.loc[:, d1]
    female_df = ddf.loc[:, d2]
    
    return male_df.iloc[0][-1], female_df.iloc[0][-1]

def get_statewise_df(state_final_df, st):
    ddf = state_final_df.copy()
    ddf = ddf[ddf['State_Code'] == st]
    ddf = ddf.groupby('State_Code').sum()
    

    d1 = [c for c in ddf.columns[2::2]]
    d2 = [c for c in ddf.columns[3::2]]
    male_df = ddf.loc[:, d1]
    female_df = ddf.loc[:, d2]

    return male_df.iloc[0][-1], female_df.iloc[0][-1]


In [94]:
dt_df = final_df.copy()
for dt in dt_df['District_Key'].unique():
    print(dt)
    mv, fv = get_districtwise_df(final_df, dt)
    dt_df.loc[dt_df['District_Key'] == dt, 'M_V'] = mv
    dt_df.loc[dt_df['District_Key'] == dt, 'F_V'] = fv

AP_Anantapur
AP_Chittoor
AP_East Godavari
AP_Guntur
AP_Krishna
AP_Kurnool
AP_Prakasam
AP_S.P.S. Nellore
AP_Srikakulam
AP_Visakhapatnam
AP_Vizianagaram
AP_West Godavari
AP_Y.S.R. Kadapa
AR_Anjaw
AR_Changlang
AR_East Kameng
AR_East Siang
AR_Kurung Kumey
AR_Lohit
AR_Lower Dibang Valley
AR_Lower Subansiri
AR_Papum Pare
AR_Tawang
AR_Tirap
AR_Upper Dibang Valley
AR_Upper Siang
AR_Upper Subansiri
AR_West Kameng
AR_West Siang
AS_Baksa
AS_Barpeta
AS_Bongaigaon
AS_Cachar
AS_Chirang
AS_Darrang
AS_Dhemaji
AS_Dhubri
AS_Dibrugarh
AS_Dima Hasao
AS_Goalpara
AS_Golaghat
AS_Hailakandi
AS_Hojai
AS_Jorhat
AS_Kamrup
AS_Kamrup Metropolitan
AS_Karbi Anglong
AS_Karimganj
AS_Kokrajhar
AS_Lakhimpur
AS_Morigaon
AS_Nagaon
AS_Nalbari
AS_Sivasagar
AS_Sonitpur
AS_Tinsukia
AS_Udalguri
BR_Araria
BR_Arwal
BR_Aurangabad
BR_Banka
BR_Begusarai
BR_Bhagalpur
BR_Bhojpur
BR_Buxar
BR_Darbhanga
BR_Gaya
BR_Gopalganj
BR_Jamui
BR_Jehanabad
BR_Kaimur
BR_Katihar
BR_Khagaria
BR_Kishanganj
BR_Lakhisarai
BR_Madhepura
BR_Madhubani
BR_Mu

In [95]:
state_final_df = final_df.copy()
for st in state_wise_pop_data['State_Code'].unique():
    if st not in state_final_df['State_Code'].unique(): continue
    mv, fv = get_statewise_df(state_final_df, st)
    state_wise_pop_data.loc[state_wise_pop_data['State_Code'] == st, 'M_V'] = mv
    state_wise_pop_data.loc[state_wise_pop_data['State_Code'] == st, 'F_V'] = fv
state_wise_pop_data = state_wise_pop_data.dropna()
state_wise_pop_data

,State_Code,TOT_M,TOT_F,M_V,F_V
1,AP,42442146,42138631,11404153.0,13593094.0
2,AR,713912,669815,398052.0,319452.0
3,AS,15939443,15266133,6961299.0,6229700.0
4,BR,54278157,49821295,16100764.0,14052584.0
5,CH,580663,474787,557241.0,420544.0
6,CT,12832895,12712303,6022745.0,5877929.0
7,DL,8987326,7800615,574185.0,393840.0
8,DN,344061,242895,459960.0,228702.0
9,GA,739140,719405,775729.0,666866.0
10,GJ,31491260,28948432,19049352.0,15655471.0


In [96]:
new_df = dt_df.copy()
new_df = new_df[['State_Code', 'District_Key', 'M_V', 'F_V', 'TOT_M', 'TOT_F']]
new_df

,State_Code,District_Key,M_V,F_V,TOT_M,TOT_F
0,AP,AP_Anantapur,893118.0,1067715.0,2064495,2016653
1,AP,AP_Chittoor,1004937.0,1201195.0,2090204,2083860
2,AP,AP_East Godavari,1125958.0,1353691.0,2569688,2584608
3,AP,AP_Guntur,1087696.0,1262966.0,2440521,2447292
4,AP,AP_Krishna,1090145.0,1223327.0,2267375,2250023
5,AP,AP_Kurnool,744366.0,990736.0,2039227,2014236
6,AP,AP_Prakasam,800299.0,947172.0,1714764,1682684
7,AP,AP_S.P.S. Nellore,783012.0,903421.0,1492974,1470583
8,AP,AP_Srikakulam,588757.0,776653.0,1341738,1361376
9,AP,AP_Visakhapatnam,1213497.0,1284911.0,2138910,2151679


In [97]:
st_df = state_wise_pop_data.copy()
st_df = st_df[['State_Code', 'M_V', 'F_V', 'TOT_M', 'TOT_F']]
st_df

,State_Code,M_V,F_V,TOT_M,TOT_F
1,AP,11404153.0,13593094.0,42442146,42138631
2,AR,398052.0,319452.0,713912,669815
3,AS,6961299.0,6229700.0,15939443,15266133
4,BR,16100764.0,14052584.0,54278157,49821295
5,CH,557241.0,420544.0,580663,474787
6,CT,6022745.0,5877929.0,12832895,12712303
7,DL,574185.0,393840.0,8987326,7800615
8,DN,459960.0,228702.0,344061,242895
9,GA,775729.0,666866.0,739140,719405
10,GJ,19049352.0,15655471.0,31491260,28948432


In [98]:
overall_pop_data['M_V'] = st_df['M_V'].sum()
overall_pop_data['F_V'] = st_df['F_V'].sum()
overall_pop_data

,Name,Level,TOT_M,TOT_F,M_V,F_V
0,India,India,623270258,587584719,253772563.0,226296569.0


In [99]:
new_df['vaccinationratio'] = new_df['F_V']/new_df['M_V']
new_df['populationratio'] = new_df['TOT_F']/new_df['TOT_M']
new_df['ratioofratios'] = new_df['vaccinationratio']/new_df['populationratio']
new_df = new_df[['District_Key', 'vaccinationratio', 'populationratio', 'ratioofratios']]
new_df.rename(columns={'District_Key':'districtid'}, inplace=True)
new_df = new_df.sort_values('ratioofratios')
new_df = new_df.set_index('districtid')
district_ratio = new_df.copy()
district_ratio

,vaccinationratio,populationratio,ratioofratios
districtid,,,
NL_Kiphire,0.531756,0.956225,0.556099
DN_Dadra and Nagar Haveli,0.441210,0.773890,0.570119
JK_Srinagar,0.539059,0.899529,0.599268
AR_Papum Pare,0.645402,0.979917,0.658629
AR_Papum Pare,0.645402,0.979917,0.658629
TN_Kancheepuram,0.652007,0.986257,0.661093
MN_Tamenglong,0.629996,0.943472,0.667742
BR_Gopalganj,0.685497,1.021047,0.671367
MN_Senapati,0.633245,0.937337,0.675579


In [100]:
st_df['vaccinationratio'] = st_df['F_V']/st_df['M_V']
st_df['populationratio'] = st_df['TOT_F']/st_df['TOT_M']
st_df['ratioofratios'] = st_df['vaccinationratio']/st_df['populationratio']
st_df = st_df[['State_Code', 'vaccinationratio', 'populationratio', 'ratioofratios']]
st_df.rename(columns={'State_Code':'stateid'}, inplace=True)
st_df = st_df.sort_values('ratioofratios')
st_df = st_df.set_index('stateid')
state_ratio = st_df.copy()
state_ratio

,vaccinationratio,populationratio,ratioofratios
stateid,,,
DN,0.497221,0.705965,0.704315
DL,0.685911,0.867957,0.790259
NL,0.752385,0.930907,0.808228
JK,0.748478,0.888562,0.842348
UP,0.775938,0.912437,0.850402
AR,0.802538,0.938232,0.855373
JH,0.825247,0.948465,0.870087
MN,0.866917,0.985140,0.879994
GA,0.859664,0.973300,0.883246


In [101]:
overall_df = overall_pop_data.copy()
overall_df['vaccinationratio'] = overall_df['F_V']/overall_df['M_V']
overall_df['populationratio'] = overall_df['TOT_F']/overall_df['TOT_M']
overall_df['ratioofratios'] = overall_df['vaccinationratio']/overall_df['populationratio']
overall_df = overall_df[['Name', 'vaccinationratio', 'populationratio', 'ratioofratios']]
overall_df.rename(columns={'Name':'id'}, inplace=True)
overall_df = overall_df.set_index('id')
overall_ratio = overall_df.copy()
overall_ratio

,vaccinationratio,populationratio,ratioofratios
id,,,
India,0.89173,0.942745,0.945887


Sorting all the ratios by final ratio 'ratioofratios'

In [102]:
district_ratio.to_csv('../output/district-vaccination-population-ratio.csv')

In [103]:
state_ratio.to_csv('../output/state-vaccination-population-ratio.csv')

In [104]:
overall_ratio.to_csv('../output/overall-vaccination-population-ratio.csv')